In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Directory where the images are stored
dir_name = '/kaggle/input/cell-images-for-detecting-malaria/cell_images/cell_images'
os.listdir(dir_name)

In [35]:
os.listdir('/kaggle/input/cell-images-for-detecting-malaria/cell_images')

In [4]:
import tensorflow as tf

In [5]:
print(tf.__version__)

In [6]:
# ImageDataGenerator class provides the abilities to process image data efficiently
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
# Define the Dataset Generator. We will use this to read files. Validation split
# determines the split into training and testing
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   validation_split = 0.2,
                                   horizontal_flip = True)

In [8]:
# Read the training set
training_set = train_datagen.flow_from_directory(dir_name,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 subset='training',
                                                 class_mode = 'binary')

In [9]:
validation_set = train_datagen.flow_from_directory(dir_name,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 subset='validation',
                                                 class_mode = 'binary')

In [10]:
training_set.class_indices

In [11]:
validation_set.class_indices

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D,Flatten, Dense

In [13]:
model = Sequential() # Create a blank model

In [14]:
# Add the first Convolution Layer and Max Pool layer
model.add(Conv2D(name='cnv_layer',
                 filters=32, 
                 kernel_size=3, 
                 activation='relu', 
                 padding='valid',
                 input_shape=[224, 224, 3]))
model.add(MaxPool2D(pool_size = 2))

In [15]:
model.add(Flatten(name='flatten')) # Add the flatten layer

In [16]:
model.add(Dense(units = 256, name='dense', activation = 'relu')) #Add the dense layer

In [17]:
model.add(Dense(name='output', units=1, activation = 'sigmoid')) # Final Output layer

In [18]:
model.summary()

In [19]:
(27+1)*32

In [20]:
# Compile the model
model.compile(optimizer='adam',
             loss = 'binary_crossentropy',
             metrics = ['accuracy'])

In [ ]:
model.fit(training_set, 
          validation_data = validation_set, 
          epochs=5) # Train the model for 5 epochs

# Transfer Learning

In [21]:
from tensorflow.keras.applications import MobileNetV2

In [22]:
mobile_net = MobileNetV2(input_shape=(224, 224, 3), include_top=False)
mobile_net.trainable = False

In [24]:
trf_model = Sequential()

In [25]:
trf_model.add(mobile_net)

In [26]:
trf_model.add(Flatten())
trf_model.add(Dense(256, activation = 'relu'))

In [27]:
trf_model.add(Dense(1, activation = 'sigmoid'))

In [28]:
trf_model.summary()

In [31]:
# Compile the model
trf_model.compile(optimizer='adam',
             loss = 'binary_crossentropy',
             metrics = ['accuracy'])

In [ ]:
trf_model.fit(training_set, validation_data = validation_set, epochs = 5)